In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

2024-05-12 14:38:54.839618: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 14:38:54.839744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 14:38:55.008444: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
import pandas as pd
loan_data = pd.read_csv("/kaggle/input/loan-approval-prediction-dataset/loan_approval_dataset.csv")
loan_data.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [4]:
# Eksik değerleri doldurma veya çıkarma
loan_data.dropna(inplace=True)

In [5]:
categorical_cols = [" education", " self_employed", " loan_status"]

In [6]:
# One-hot encoding
data_encoded = pd.get_dummies(loan_data, columns=categorical_cols, drop_first=True)

In [7]:
print(data_encoded.head())

   loan_id   no_of_dependents   income_annum   loan_amount   loan_term  \
0        1                  2        9600000      29900000          12   
1        2                  0        4100000      12200000           8   
2        3                  3        9100000      29700000          20   
3        4                  3        8200000      30700000           8   
4        5                  5        9800000      24200000          20   

    cibil_score   residential_assets_value   commercial_assets_value  \
0           778                    2400000                  17600000   
1           417                    2700000                   2200000   
2           506                    7100000                   4500000   
3           467                   18200000                   3300000   
4           382                   12400000                   8200000   

    luxury_assets_value   bank_asset_value   education_ Not Graduate  \
0              22700000            8000000        

In [8]:
X = data_encoded.drop(" loan_status_ Rejected", axis=1) 
y = data_encoded[" loan_status_ Rejected"]

In [9]:
# Boolean sütunların adlarını kaldırma
X.columns = [col.split('_')[0] for col in X.columns if '_' in col]

In [10]:
# Veriyi numpy dizilerine dönüştürme
X = X.values
y = y.values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# Özellikleri ölçeklendirme
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# LSTM modeli oluşturma
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [14]:
# Modeli derleme
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
# Modeli eğitme
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.5880 - loss: 0.6716 - val_accuracy: 0.6223 - val_loss: 0.6613
Epoch 2/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6188 - loss: 0.6512 - val_accuracy: 0.6354 - val_loss: 0.6506
Epoch 3/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.6284 - loss: 0.6409 - val_accuracy: 0.6559 - val_loss: 0.6300
Epoch 4/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.6738 - loss: 0.6043 - val_accuracy: 0.7233 - val_loss: 0.5726
Epoch 5/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7226 - loss: 0.5479 - val_accuracy: 0.7233 - val_loss: 0.5390
Epoch 6/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7742 - loss: 0.4588 - val_accuracy: 0.7818 - val_loss: 0.4559
Epoch 7/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7641 - loss: 0.4689 - val_accuracy: 0.7892 - val_loss: 0.4349
Epoch 8/50
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7905 - loss: 0.4310 - val_accuracy: 0.8316 - v

In [16]:
# Modeli değerlendirme
accuracy = model.evaluate(X_test, y_test)[1]
print("Test doğruluğu:", accuracy)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9552 - loss: 0.1087
Test doğruluğu: 0.9531615972518921


In [17]:
# Model tahminlerini ikili sınıflara dönüştürme
threshold = 0.5
y_pred_classes = (y_pred >= threshold).astype(int)

# Confusion matrix'i hesaplama
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Renkli görselleştirme
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

NameError: name 'y_pred' is not defined

In [ ]:
from sklearn.metrics import classification_report

# Classification report'u hesaplama
report = classification_report(y_true, y_pred_classes)

print(report)